In [7]:
#Importing align package from biopython
from Bio import Align
from Bio.Align import substitution_matrices

#Creating a dictionary with MSI gene names as keys and lists to store tuples of alignment scores, alignments, SRR file names, 
#SRR positions, CpG island counts, and mononucleotide sequences
mylist = {'nacad': [], 'xirp1': [], '5730596b20rik': [], 'rif1': [], 'maz': [],
          'hic1': [], 'sdccag1': [], 'tmem107': [], 'srcin1': [], 'marcks': [],
          'senp6': [], 'phactr4': [], 'chrnb2': []}
#Creating a list of SRR file names that will be searched for FSPs
mySRRList = ['SRR14676234', 'SRR14676248', 'SRR14676209', 'SRR14676235', 'SRR14676227']
for SRRfile in mySRRList:
    #Reading in the files that contain the 7 amino acid long peptide sequences alongside the file that contains the 
    #corresponding chromsome positions, CpG island counts, and mononucleotide sequences for those specific peptide sequences
    #with a count variable to establish which chromsome positions, CpG island counts, and mononucleotide sequences match the
    #the specific peptide sequence being observed in the file
    f = open(SRRfile + '/' + SRRfile + 'format3.txt', 'r')
    f2 = open(SRRfile + '/' + SRRfile + 'format2.txt', 'r')
    lines = f2.readlines()
    count = 0
    for row in f:
        #A dictionary to contain the FSP bait sequences for the 13 MSI genes used in the study
        fspSequences = {'nacad': 'VIYAPPP', 'xirp1': 'GKGPGGPP', '5730596b20rik': 
                       'GTLPPPP', 'rif1': 'AHTDKKKK', 'maz': 'PCTLLAP', 'hic1': 'DRTFPSPP',
                       'sdccag1': 'EAPKGKKK', 'tmem107': 'TQYFGMGG', 'srcin1': 'DEGMWPP', 'marcks': 'SSETPKKK',
                       'senp6': 'VKCSMKKK', 'phactr4': 'PWKWRKKK', 'chrnb2': 'VRTRPSPP'}
        #Generating alignments using a BLOSUM62 scoring matrix for each 7 amino acid long peptide sequence crossed with each 
        #FSP bait sequence in the dictionary containing the 13 MSI genes
        for key in fspSequences:
            aligner = Align.PairwiseAligner()
            aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
            alignments = aligner.align(str(fspSequences[key]), row[1:len(row)-2])
            alignments = list(alignments)
            alignment = alignments[0]
            
            line = lines[count].split('\t')
            srrposition = line[0]
            cpgcount = line[1]
            mononucsequence = line[2]
            mylist[key].append((alignment.score, alignment, SRRfile, srrposition, cpgcount, mononucsequence))
        count+=1
    f2.close()
    f.close()
    print("Done.")

#Find the 100 highest scoring alignments for each MSI gene in the 'mylist' dictionary
for key in mylist:
    while(len(mylist[key]) > 100):
        min = float('inf')
        for entry in mylist[key]:
            if(entry[0] < min):
                min = entry[0]
        for entry in mylist[key]:
            if(entry[0] == min):
                mylist[key].remove(entry)
                break
    print(key, "done.")
#Generating a new file to store each possible neoantigen replacement found denoted by its MSI gene name, MSI value generated from 
#the number of CpG islands with a 1000x multiplier and the length of the longest mononucleotide sequence found 20 nucleotide bases 
#upstream of the CDS with a multiplier of 100x, the upper alignment, aswell as the lower alignment, SRR file name, chromosome position, 
#and mononucleotide sequence
f1 = open('fileOutScore.txt', 'a')
for key in mylist:
    for entry in mylist[key]:
        MSI_Val = entry[0] + (int(entry[4])*1000) + (len(entry[5])*100)
        f1.write(str(key) + '\t' + str(MSI_Val) + '\t' + str(entry[1]).split('\n')[0] + '\t' + str(entry[1]).split('\n')[2] + '\t' + str(entry[2]) + '\t' + str(entry[3]) + '\t' + str(entry[5]))
f1.close()

Done.
Done.
Done.
Done.
Done.
nacad done.
xirp1 done.
5730596b20rik done.
rif1 done.
maz done.
hic1 done.
sdccag1 done.
tmem107 done.
srcin1 done.
marcks done.
senp6 done.
phactr4 done.
chrnb2 done.
